# <B> Setup for SageMaker pipleline examples </B>

## AutoReload

In [54]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Set roles

In [55]:
from sagemaker import get_execution_role

In [56]:
strSageMakerRoleName = get_execution_role().rsplit('/', 1)[-1]
print (f"SageMaker Execution Role Name: {strSageMakerRoleName}")

SageMaker Execution Role Name: AmazonSageMaker-ExecutionRole-20221206T163436


### 2.1 Attach IAM polich to sagemaker execution role (<b>with console</b>)
> step 1. IAM console 로 이동    
>
> step 2. 왼쪽 네비게이터에서  "Role" 선택  
> ![nn](images/Role.png)  
>
> step 3. SageMaker Execution Role 검색 후 role 선택 (상위 cell output 참조)  
> ![nn](images/search-by-rolename.png)  
>
> step 4. "attach policies" 메뉴 선택 
> ![nn](images/attach-policy-menu.png)  
>
> step 5. "IAMFullAccess" policy 검색 후 attach 
> ![nn](images/attach-policy.png) 

In [57]:
from utils.iam import iam_handler

In [58]:
iam = iam_handler()

### 2.2 Attach policy - sagemaker execution role
> SSM  
> EC2ContainerRegistry <BR>
> S3

In [59]:
listPolicyArn=[
    "arn:aws:iam::aws:policy/AmazonSSMFullAccess",
    "arn:aws:iam::aws:policy/AmazonEC2ContainerRegistryFullAccess",
    "arn:aws:iam::aws:policy/AmazonS3FullAccess",
]
for strPolicyArn in listPolicyArn: iam.attach_policy(strSageMakerRoleName, strPolicyArn)

## 2. Set default parameters

In [60]:
import boto3
import sagemaker
from utils.ssm import parameter_store

In [61]:
strRegionName=boto3.Session().region_name
pm = parameter_store(strRegionName)

In [62]:
strPrefix = "SM-PIPELINE-HAND-ON-"

In [69]:
pm.put_params(key="PREFIX", value=strPrefix, overwrite=True)
pm.put_params(key="".join([strPrefix, "REGION"]), value=strRegionName, overwrite=True)
pm.put_params(key="".join([strPrefix, "BUCKET"]), value=sagemaker.Session().default_bucket(), overwrite=True)
pm.put_params(key="".join([strPrefix, "SAGEMAKER-ROLE-ARN"]), value=get_execution_role(), overwrite=True)
pm.put_params(key="".join([strPrefix, "ACCOUNT-ID"]), value=boto3.client("sts").get_caller_identity().get("Account"), overwrite=True)

'Store suceess'

## 3. Data preparation

In [70]:
import os

In [71]:
strSrcPath = "s3://amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"
strTargetPath = f"s3://{pm.get_params(key=strPrefix + 'BUCKET')}/reviews-helpfulness-pipeline/data/reviews.tsv.gz"

In [72]:
strQuery = "".join(["aws s3 cp ", strSrcPath, " ", strTargetPath])
strResponse = os.popen(strQuery).read()

In [73]:
pm.put_params(key="".join([strPrefix, "REVIEW-DATA-PATH"]), value=strTargetPath, overwrite=True)

'Store suceess'

In [74]:
print (f'input data: {pm.get_params(key=strPrefix + "REVIEW-DATA-PATH")}')

input data: s3://sagemaker-ap-northeast-2-419974056037/reviews-helpfulness-pipeline/data/reviews.tsv.gz
